In [3]:
%cd yolo5

[Errno 2] No such file or directory: 'yolo5'
/home/sayedgomaa/PyProject/Gradution Project/Yolo5


In [4]:
!pip install -r yolov5/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 905.3 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 998.8 kB/s eta 0:00:00MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 941.8 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 KB 948.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [ ]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()
    
    # Initialise the info dict 
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name 
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
        
        # Get details of the bounding box 
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict


In [ ]:
print(extract_info_from_xml('Val_Anno/000458237.xml'))

In [ ]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = { '5Egp':0, '10Egp':1, '20Egp':2,'50Egp':3,'100Egp':4,'200Egp':5}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("labels/train", info_dict["filename"].replace("jpg", "txt"))
    
    # Save the annotation to dis
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
# Get the annotations
annotations = [os.path.join('labels/train', x) for x in os.listdir('labels/train') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)

In [ ]:
annotations = [os.path.join('labels/train', x) for x in os.listdir('labels/train') if x[-3:] == "txt"]

In [ ]:
print(annotations)

In [ ]:
# Get the annotations
annotations = [os.path.join('Train_Anno', x) for x in os.listdir('Train_Anno') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
annotations = [os.path.join('Train_Anno', x) for x in os.listdir('Train_Anno') if x[-3:] == "txt"]

100%|██████████| 8996/8996 [03:22<00:00, 44.52it/s] 


In [ ]:
# annotations = [os.path.join('Val_Image', x) for x in os.listdir('Val_Anno') if x[-3:] == "txt"]
print(annotations)

In [ ]:
random.seed(0)

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h 
    
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
        
        plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(annotations)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]

#Get the corresponding image file
image_file = annotation_file.replace("Val_Anno", "Val_Image").replace("txt", "jpg")
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

In [ ]:
!wget -P data/ https://gist.githubusercontent.com/ayooshkathuria/bcf7e3c929cbad445439c506dba6198d/raw/f437350c0c17c4eaa1e8657a5cb836e65d8aa08a/road_sign_data.yaml


In [12]:
%cd Yolo5

/home/sayedgomaa/PyProject/Gradution Project/Yolo5


In [13]:
! ls

cfg.yaml  clss.txt  yolov5  yolov5.ipynb


In [ ]:
!python yolov5/train.py --img 640 --cfg yolov5s.yaml --hyp yolov5/data/hyps/hyp.scratch-high.yaml --batch 32 --epochs 10 --data cfg.yaml --weights yolov5/runs/train/yolo_road_det40/weights/best.pt --workers 24 --name yolo_road_det

In [2]:
!python3 yolov5/detect.py --source yolov5/tst --weights yolov5/runs/train/yolo_road_det42/weights/best.pt --conf 0.25 --name  cfg.yaml

detect: weights=['yolov5/runs/train/yolo_road_det42/weights/best.pt'], source=yolov5/tst, data=yolov5/cfg.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=cfg.yaml, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2022-3-3 torch 1.11.0+cu102 CPU

Fusing layers... 
Model Summary: 213 layers, 7026307 parameters, 0 gradients
image 1/3 /home/sayedgomaa/PyProject/Gradution Project/Yolo5/yolov5/tst/1.jpeg: 640x288 1 10Egp, Done. (0.139s)
image 2/3 /home/sayedgomaa/PyProject/Gradution Project/Yolo5/yolov5/tst/2.jpeg: 640x288 1 100Egp, Done. (0.147s)
image 3/3 /home/sayedgomaa/PyProject/Gradution Project/Yolo5/yolov5/tst/3.jpg: 480x640 1 5Egp, 1 20Egp, Done. (0.181s)
Speed: 1.0ms pre-process, 155.6ms inference, 1.0ms NMS p

In [ ]:
! python yolov5/detect.py --weights yolov5/runs/train/yolo_road_det40/weights/best.pt --source 0              # webcam

In [ ]:
!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2
img = cv2.imread('logo.png', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

In [ ]:
import tensorflow as tf
print(tf.__version__)
!PYTHONPATH=. python3 /content/yolov5/models/tf.py --weight
/content/yolov5/runs/train/exp2/weights/best.pt --cfg models/yolov5s.yaml --img 416 --source /content/FaceMaskDataset/images/train

In [4]:

!pip install tensorflow-addons
!pip install onnx-tf
!pip install tensorflow==2.3.0
 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!python yolov5/export.py --weights yolov5/runs/train/yolov5s.pt --img 416 --batch 1 # export at 640x640 with batch size 1


export: data=yolov5/data/coco128.yaml, weights=['yolov5/runs/train/yolov5s.pt'], imgsz=[416], batch_size=1, device=cpu, half=False, inplace=False, train=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 v6.1-11-g63ddb6f torch 1.11.0+cu113 CPU

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Traceback (most recent call last):
  File "yolov5/export.py", line 559, in <module>
    main(opt)
  File "yolov5/export.py", line 554, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "yolov5/export.py", line 474, in run
    y = model(im)  # dry runs
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    retu

In [6]:
import tensorflow_addons as tfa
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


ImportError: ignored

In [ ]:
!ls

In [7]:

base_model = onnx.load('yolov5/runs/train/yolov5s.onnx')
to_tf = prepare(base_model)
to_tf.export_graph("yolov5/runs/train/exp3")

FileNotFoundError: ignored

In [ ]:
converter = tf.compat.v1.lite.TFLiteConverter.from_saved_model('yolov5/runs/train/exp3')


In [ ]:
experimental_new_converter=False

In [ ]:
tflite_model = converter.convert() #just FYI: this step could go wrong and your notebook instance could crash.

INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False


In [ ]:
!tflite_convert --saved_model_dir=yolov5/runs/train/exp2 --output_file=yolov5/runs/train/exp/model_v12_24.tflite --experimental_new_converter=False


In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import accuracy_score

In [ ]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size

In [ ]:
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')

In [ ]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"

In [8]:
# Converting a SavedModel to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_saved_model('/content/gdrive/MyDrive/yolo/yolov5/runs/train/exp3')


NameError: ignored

In [ ]:
tflite_model = converter.convert()

INFO:absl:Using experimental converter: If you encountered a problem please file a bug. You can opt-out by setting experimental_new_converter=False
